# Cortex Compliance AI - Fine-Tuning

Dataset already uploaded to HuggingFace. Just run this notebook on Colab with T4 GPU.

## Steps:
1. Runtime → Change runtime type → **T4 GPU**
2. Run all cells

In [ ]:
# Step 1: Install dependencies (tested stable versions)
!pip uninstall -y transformers accelerate peft bitsandbytes trl -q 2>/dev/null
!pip install -q transformers==4.41.0 accelerate==0.30.1 peft==0.11.1 bitsandbytes==0.43.1 trl==0.8.6
!pip install -q datasets huggingface_hub sentencepiece

print("✅ Installed! Now: Runtime → Restart runtime → Run from Step 2")

In [ ]:
# Step 2: Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Step 3: Load dataset from HuggingFace (already uploaded!)
from datasets import load_dataset

dataset = load_dataset("maaninder/cortex-compliance-data", split="train")
print(f"✅ Loaded {len(dataset)} training examples from HuggingFace")
print(dataset[0]["text"][:200] + "...")

In [ ]:
# Step 4: Load model with 4-bit quantization
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0},
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, LoraConfig(
    r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
))

print("✅ Model loaded with LoRA")
model.print_trainable_parameters()

In [ ]:
# Step 5: Train
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    args=TrainingArguments(
        output_dir="./output",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="paged_adamw_8bit",
        save_strategy="no",
        report_to="none",
    ),
)

print("🚀 Training started...")
trainer.train()
print("✅ Training complete!")

In [ ]:
# Step 6: Save to HuggingFace Hub
from huggingface_hub import notebook_login
notebook_login()  # Enter your token when prompted

model.push_to_hub("maaninder/cortex-compliance-ai")
tokenizer.push_to_hub("maaninder/cortex-compliance-ai")
print("✅ Model saved to: https://huggingface.co/maaninder/cortex-compliance-ai")

In [ ]:
# Step 7: Test the model
test_prompt = "### Instruction:\nGenerate a Personal Data Processing Policy for ООО Тест (INN: 1234567890)\n\n### Response:\n"
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

model.eval()
outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Done!

Model saved to: https://huggingface.co/maaninder/cortex-compliance-ai

**Tested versions:**
- transformers==4.41.0
- peft==0.11.1
- bitsandbytes==0.43.1
- trl==0.8.6